In [1]:
from itertools import product

import sys
import os

# Ruta absoluta a la carpeta ra√≠z (donde est√° la carpeta "scripts")
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Sub√≠s un nivel
sys.path.append(root_path)

param_grid = {
    'k_neighbors': [3, 5, 7],
    'radio_densidad': [0.8, 1.0, 1.2],
    'percentil_dist': [50, 75, 90]
}


In [2]:
# ============================
# CARGA Y PREPROCESAMIENTO
# ============================

import pandas as pd
import numpy as np

# Ruta del dataset
ruta_ecoli = '../datasets/ecoli/ecoli.data'

# Leer el dataset con separaci√≥n por espacios
df = pd.read_csv(ruta_ecoli, header=None, delim_whitespace=True)

# Separar caracter√≠sticas (X) y etiquetas (y)
X = df.iloc[:, 1:8].values
y = df.iloc[:, 8].values

# Binarizar: 'cp' es la clase minoritaria
y_bin = np.where(y == 'cp', 1, 0)

# Separar para an√°lisis si se requiere
X_min = X[y_bin == 1]
X_maj = X[y_bin == 0]

# Verificar distribuci√≥n
print("‚úÖ Dataset cargado.")
print("üî¥ Instancias minoritarias (cp):", len(X_min))
print("‚ö™ Instancias mayoritarias:", len(X_maj))


‚úÖ Dataset cargado.
üî¥ Instancias minoritarias (cp): 143
‚ö™ Instancias mayoritarias: 193


C:\Users\User\AppData\Local\Temp\ipykernel_3824\3699374835.py:12: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(ruta_ecoli, header=None, delim_whitespace=True)


In [3]:
from scripts.evaluacion import evaluar_sampler_holdout, evaluar_sampler_completo
from scripts.pc_smote import PCSMOTE


param_grid = [
    (k, radio, p_dist)
    for k in [3, 5, 7]
    for radio in [0.5, 1.0, 1.5]
    for p_dist in [50, 75, 90]
]

resultados = []

for k, radio, p_dist in param_grid:
    print(f"\nüîç Evaluando: k={k}, radio={radio}, percentil_dist={p_dist}")
    
    sampler = PCSMOTE(
        k_neighbors=k,
        radio_densidad=radio,
        percentil_dist=p_dist,
        random_state=42
    )
    
    metricas = evaluar_sampler_holdout(
        nombre="PC-SMOTE",
        sampler_class=PCSMOTE,
        X=X,
        y_bin=y_bin,
        n_iter=5,
        k_neighbors=k,
        radio_densidad=radio,
        percentil_dist=p_dist
    )
    
    resultados.append({
        "k": k,
        "radio": radio,
        "percentil_dist": p_dist,
        **metricas
    })

df_resultados = pd.DataFrame(resultados)
print(df_resultados.columns)
df_resultados = df_resultados.sort_values(by="mean_f1", ascending=False)
df_resultados.reset_index(drop=True, inplace=True)



üîç Evaluando: k=3, radio=0.5, percentil_dist=50
üìå Total muestras minoritarias: 100
üìå Total muestras mayoritarias: 135
üìä Riesgo - media: 0.0333 | min: 0.0000 | max: 1.0000
üìä Densidad - media: 1.0000 | p25: 1.0000 | p50: 1.0000 | p75: 1.0000
üîé Muestras con riesgo ‚àà [0.4, 0.6]: 0
üîé Muestras con densidad > 0: 100
‚úÖ Muestras seleccionadas para sobremuestreo (intersecci√≥n): 0
‚ùå Muestras descartadas (ruido o baja densidad): 100
‚ö†Ô∏è No se encontraron muestras v√°lidas para sobremuestreo. Devolviendo conjunto original.
üìå Total muestras minoritarias: 100
üìå Total muestras mayoritarias: 135
üìä Riesgo - media: 0.0400 | min: 0.0000 | max: 1.0000
üìä Densidad - media: 1.0000 | p25: 1.0000 | p50: 1.0000 | p75: 1.0000
üîé Muestras con riesgo ‚àà [0.4, 0.6]: 0
üîé Muestras con densidad > 0: 100
‚úÖ Muestras seleccionadas para sobremuestreo (intersecci√≥n): 0
‚ùå Muestras descartadas (ruido o baja densidad): 100
‚ö†Ô∏è No se encontraron muestras v√°lidas para sob

In [4]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

clasificadores = {
    "RandomForest": RandomForestClassifier,
    "XGBoost": XGBClassifier,
    "SVM": lambda **kwargs: SVC(probability=True, **kwargs),
    "LogisticRegression": LogisticRegression,
    "MLP": MLPClassifier,
    "kNN": KNeighborsClassifier
}

# combinaci√≥n √≥ptima hallada
sampler_args = {
    "k_neighbors": 3,
    "radio_densidad": 0.5,
    "percentil_dist": 75
}

resultados = []

for nombre_clf, clf_class in clasificadores.items():
    print(f"\nüîç Evaluando clasificador: {nombre_clf}")
    metricas = evaluar_sampler_holdout(
        nombre=f"PC-SMOTE + {nombre_clf}",
        sampler_class=PCSMOTE,
        X=X,
        y_bin=y_bin,
        n_iter=5,
        modelo=clf_class,
        **sampler_args
    )
    resultados.append(metricas)

df_modelos = pd.DataFrame(resultados).sort_values(by="mean_f1", ascending=False)



üîç Evaluando clasificador: RandomForest
üìå Total muestras minoritarias: 100
üìå Total muestras mayoritarias: 135
üìä Riesgo - media: 0.0333 | min: 0.0000 | max: 1.0000
üìä Densidad - media: 1.0000 | p25: 1.0000 | p50: 1.0000 | p75: 1.0000
üîé Muestras con riesgo ‚àà [0.4, 0.6]: 0
üîé Muestras con densidad > 0: 100
‚úÖ Muestras seleccionadas para sobremuestreo (intersecci√≥n): 0
‚ùå Muestras descartadas (ruido o baja densidad): 100
‚ö†Ô∏è No se encontraron muestras v√°lidas para sobremuestreo. Devolviendo conjunto original.
üìå Total muestras minoritarias: 100
üìå Total muestras mayoritarias: 135
üìä Riesgo - media: 0.0400 | min: 0.0000 | max: 1.0000
üìä Densidad - media: 1.0000 | p25: 1.0000 | p50: 1.0000 | p75: 1.0000
üîé Muestras con riesgo ‚àà [0.4, 0.6]: 0
üîé Muestras con densidad > 0: 100
‚úÖ Muestras seleccionadas para sobremuestreo (intersecci√≥n): 0
‚ùå Muestras descartadas (ruido o baja densidad): 100
‚ö†Ô∏è No se encontraron muestras v√°lidas para sobremuestr

C:\Users\User\AppData\Roaming\Python\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\User\AppData\Roaming\Python\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


üìå Total muestras minoritarias: 100
üìå Total muestras mayoritarias: 135
üìä Riesgo - media: 0.0633 | min: 0.0000 | max: 1.0000
üìä Densidad - media: 1.0000 | p25: 1.0000 | p50: 1.0000 | p75: 1.0000
üîé Muestras con riesgo ‚àà [0.4, 0.6]: 0
üîé Muestras con densidad > 0: 100
‚úÖ Muestras seleccionadas para sobremuestreo (intersecci√≥n): 0
‚ùå Muestras descartadas (ruido o baja densidad): 100
‚ö†Ô∏è No se encontraron muestras v√°lidas para sobremuestreo. Devolviendo conjunto original.
üìå Total muestras minoritarias: 100
üìå Total muestras mayoritarias: 135
üìä Riesgo - media: 0.0433 | min: 0.0000 | max: 1.0000
üìä Densidad - media: 1.0000 | p25: 1.0000 | p50: 1.0000 | p75: 1.0000
üîé Muestras con riesgo ‚àà [0.4, 0.6]: 0
üîé Muestras con densidad > 0: 100
‚úÖ Muestras seleccionadas para sobremuestreo (intersecci√≥n): 0
‚ùå Muestras descartadas (ruido o baja densidad): 100
‚ö†Ô∏è No se encontraron muestras v√°lidas para sobremuestreo. Devolviendo conjunto original.


C:\Users\User\AppData\Roaming\Python\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\User\AppData\Roaming\Python\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


üìå Total muestras minoritarias: 100
üìå Total muestras mayoritarias: 135
üìä Riesgo - media: 0.0667 | min: 0.0000 | max: 0.6667
üìä Densidad - media: 1.0000 | p25: 1.0000 | p50: 1.0000 | p75: 1.0000
üîé Muestras con riesgo ‚àà [0.4, 0.6]: 0
üîé Muestras con densidad > 0: 100
‚úÖ Muestras seleccionadas para sobremuestreo (intersecci√≥n): 0
‚ùå Muestras descartadas (ruido o baja densidad): 100
‚ö†Ô∏è No se encontraron muestras v√°lidas para sobremuestreo. Devolviendo conjunto original.

üîç Evaluando clasificador: kNN
üìå Total muestras minoritarias: 100
üìå Total muestras mayoritarias: 135
üìä Riesgo - media: 0.0333 | min: 0.0000 | max: 1.0000
üìä Densidad - media: 1.0000 | p25: 1.0000 | p50: 1.0000 | p75: 1.0000
üîé Muestras con riesgo ‚àà [0.4, 0.6]: 0
üîé Muestras con densidad > 0: 100
‚úÖ Muestras seleccionadas para sobremuestreo (intersecci√≥n): 0
‚ùå Muestras descartadas (ruido o baja densidad): 100
‚ö†Ô∏è No se encontraron muestras v√°lidas para sobremuestreo. Devol

C:\Users\User\AppData\Roaming\Python\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
